In [1]:
%load_ext dotenv
%dotenv

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from utils import to_timestamp_df

spark = SparkSession.builder.appName("Trip Data").getOrCreate()
src_path = "s3://aws-data-analytics-workshops/shared_datasets/tripdata/"
ny_taxi = spark.read.option("inferSchema", "true").option("header", "true").csv(src_path)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/share/aws/emr/emrfs/lib/slf4j-log4j12-1.7.12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
ny_taxi = to_timestamp_df(ny_taxi, ['lpep_pickup_datetime', 'lpep_dropoff_datetime'])
ny_taxi.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)



In [4]:
ny_taxi = ny_taxi.withColumn('ride_duration', col('lpep_dropoff_datetime').cast('long') - col('lpep_pickup_datetime').cast('long'))
ny_taxi.select('ride_duration').summary().show()

+-------+-----------------+
|summary|    ride_duration|
+-------+-----------------+
|  count|            20000|
|   mean|          1602.87|
| stddev|8265.798653153171|
|    min|                0|
|    25%|              360|
|    50%|              660|
|    75%|             1020|
|    max|            86400|
+-------+-----------------+

